In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
input_width=28
input_height=28
input_channel=1
input_pixels=784
n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2
n_hidden=1024
n_out=10
input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_cov2

In [9]:
weights={
    "wc1":tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channel,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([conv2_k,conv2_k,n_cov1,n_conv2])),
    "wh1":tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.random_normal([n_hidden,n_out]))
}
biases={
    "bc1":tf.Variable(tf.random_normal([n_conv1])),
    "bc2":tf.Variable(tf.random_normal([n_conv2])),
    "bh1":tf.Variable(tf.random_normal([n_hidden])),
    "bo":tf.Variable(tf.random_normal([n_out]))
}

In [20]:
def cnn(x,weights,biases):
    x=tf.reshape(x,shape=[-1,input_height,input_width,input_channel])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool=maxpooling(conv1,max_pool1_k)
    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool=maxpooling(conv2,max_pool2_k)
    
    hidden_input=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
    hidden_output_before_activation=tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output=tf.nn.relu(hidden_output_before_activation)
    output=tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [26]:
def conv(x,weights,biases,strides=1):
    out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,biases)
    out=tf.nn.relu(out)
    return out
def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [30]:
x=tf.placeholder("float",[None,input_pixels])
y=tf.placeholder(tf.int32,[None,n_out])
pred=cnn(x,weights,biases)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [33]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [36]:
batch_size=100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y})
        total_cost+=c
    print(total_cost)  

923932.561966896
34775.42513333261
19045.241263349042
12895.427157424278
12791.481078686193
14377.163433325015
11488.830873816174
11700.188747080565
9275.955282036708
7195.353482571838
9276.609441802226
6093.384251172133
5616.635976124999
5262.108468522407
5755.863429695888
3606.95079947263
3814.668480481603
3011.575597607344
3538.611477649814
3464.7980661407364
3536.551879877795
2474.653305345345
2138.226689249903
3171.764438214948
2314.4100761859327


In [51]:
predictions=tf.argmax(pred,1)
correct_labels=tf.argmax(y,1)
correct_predictions=tf.equal(predictions,correct_labels)
predictions,correct_predictions=sess.run([predictions,correct_predictions],feed_dict={x:mnist.test.images,y:mnist.test.labels})
correct_predictions.sum()

9866